In [16]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

2022-03-14 17:06:52.794213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 17:06:52.794226: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
weather_event_cat = {
    'inconnu': 1, 'pluie faible': 2, 'ciel clair': 3, 'brouillard faible': 4, 'pluie': 5, 'brouillard': 6,
    'neige faible': 7, 'pluie forte': 8, 'neige': 9, 'brouillard fort': 10
}

atmo_cat = {'bon': 1, 'moyen': 2, 'dégradé': 3, 'mauvais': 4, "très mauvais": 5, "extrêmement mauvais": 6}

wind_dir_cat = {
    'SO': 1, 'O': 2, 'SSO': 3, 'N': 4, 'S': 5, 'NE': 6, 'OSO': 7, 'NNO': 8, 'ONO': 9, 'ENE': 10, 'E': 11,
    'NNE': 12, 'NO': 13, 'SSE': 14, 'SE': 15, 'ESE': 16
}

In [20]:
df = pd.read_csv("https://raw.githubusercontent.com/thibaultrichel/citeos-air-quality/main/data/final/merged-final.csv", sep=';').dropna().drop("day", axis=1)
df.head(25)

,date,PM10,PM25,NO2,SO2,NO,NOX,O3,ATMO,temp,wind_speed,wind_dir,hum,press,weather_event
0,2019-01-01 01:00:00,5.9,9.2,18.6,2.4,1.3,20.7,41.8,bon,8.2,3.7,ONO,84.0,1036.0,inconnu
1,2019-01-01 02:00:00,5.4,10.3,19.7,2.3,1.4,21.9,39.8,moyen,7.9,3.7,NNO,90.0,1036.2,inconnu
2,2019-01-01 03:00:00,8.6,12.9,24.3,2.1,0.7,25.4,32.8,moyen,7.7,3.7,O,88.0,1035.8,inconnu
3,2019-01-01 04:00:00,10.2,12.5,25.4,2.6,1.0,27.0,36.6,moyen,7.9,3.7,ONO,82.0,1035.4,inconnu
4,2019-01-01 05:00:00,11.1,6.9,18.2,3.1,0.6,19.2,48.9,bon,8.0,3.7,O,81.0,1034.8,inconnu
5,2019-01-01 06:00:00,10.7,6.1,26.3,3.2,1.6,28.8,34.8,bon,8.3,3.7,ONO,80.0,1034.9,inconnu
6,2019-01-01 07:00:00,15.6,10.4,31.2,3.0,2.7,35.3,36.3,moyen,8.4,3.7,ONO,79.0,1035.0,inconnu
7,2019-01-01 08:00:00,18.2,11.8,25.1,3.1,2.8,29.4,39.8,moyen,8.5,5.6,NO,79.0,1035.0,inconnu
8,2019-01-01 09:00:00,12.0,8.6,13.6,3.2,1.8,16.3,55.5,moyen,8.4,7.4,NNO,77.0,1035.0,inconnu
9,2019-01-01 10:00:00,15.4,7.3,12.8,3.5,1.3,14.8,60.2,moyen,8.6,5.6,NO,76.0,1034.9,inconnu


In [19]:
df = pd.read_csv("https://raw.githubusercontent.com/thibaultrichel/citeos-air-quality/main/data/final/merged-final.csv", sep=';').dropna().drop("day", axis=1)
df["wind_dir_cat"] = df.wind_dir.apply(lambda x: wind_dir_cat[x])
df["weather_event_cat"] = df.weather_event.apply(lambda x: weather_event_cat[x])
df["atmo_cat"] = df.ATMO.apply(lambda x: atmo_cat[x])
df = df.drop(["ATMO", "weather_event", "wind_dir"], axis=1)
df

,date,PM10,PM25,NO2,SO2,NO,NOX,O3,temp,wind_speed,hum,press,wind_dir_cat,weather_event_cat,atmo_cat
20,2019-01-01 21:00:00,17.3,10.1,37.2,3.1,3.7,42.9,23.9,8.6,5.6,90.0,1035.7,4,1,2
21,2019-01-01 22:00:00,13.8,11.3,30.8,3.3,1.7,33.4,26.8,8.6,3.7,90.0,1036.0,8,1,2
22,2019-01-01 23:00:00,13.9,9.9,23.7,3.3,1.6,26.2,36.9,8.0,11.1,79.0,1036.6,12,1,1
23,2019-01-02 00:00:00,15.7,7.5,13.2,4.1,1.7,15.8,55.8,7.5,9.3,72.0,1037.3,12,1,2
24,2019-01-02 01:00:00,16.5,7.5,11.5,1.0,0.9,12.8,59.4,6.6,7.4,74.0,1037.7,4,1,2
25,2019-01-02 02:00:00,13.7,7.8,9.1,0.7,0.2,9.3,60.1,6.2,7.4,76.0,1038.2,4,1,2
26,2019-01-02 03:00:00,12.7,6.9,9.6,1.0,0.2,9.9,61.0,6.0,7.4,78.0,1038.7,4,1,2
27,2019-01-02 04:00:00,14.5,8.3,10.0,1.1,0.3,10.6,59.0,5.8,7.4,78.0,1038.7,12,1,2
28,2019-01-02 05:00:00,13.1,7.0,19.7,1.0,1.0,21.2,52.8,5.7,5.6,72.0,1038.9,4,1,2
30,2019-01-02 07:00:00,22.4,9.2,39.8,0.8,6.0,49.0,41.4,5.8,5.6,70.0,1039.7,4,1,2


In [4]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size):
    data, labels = [], []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = list(range(i-history_size, i))
        data.append(dataset[indices])
        labels.append(target[i+target_size])
    return np.array(data), np.array(labels)

In [5]:
values = df.drop("date", axis=1).values
values

array([[ 5.9,  9.2, 18.6, ...,  9. ,  1. ,  1. ],
       [ 5.4, 10.3, 19.7, ...,  8. ,  1. ,  2. ],
       [ 8.6, 12.9, 24.3, ...,  2. ,  1. ,  2. ],
       ...,
       [11.9,  8.4, 25.8, ...,  3. ,  3. ,  1. ],
       [11.8,  8.8, 31.1, ...,  3. ,  3. ,  1. ],
       [14.1, 10.5, 35.9, ...,  3. ,  3. ,  2. ]])

In [6]:
n_past = 120
n_future = 12
train_size = int(values.shape[0] * 0.75)

X_train, y_train = multivariate_data(
    dataset=values,
    target=values[:, 13],
    start_index=0,
    end_index=train_size,
    history_size=n_past,
    target_size=n_future
)

X_train.shape, y_train.shape

((19088, 120, 14), (19088,))

In [18]:
X_val = np.array(values[train_size:train_size + n_past])
X_val = np.expand_dims(X_val, axis=0)
X_val.shape

(1, 120, 14)

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer="adam", loss="mse", metrics=["mse"])

2022-03-14 17:07:47.463409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-14 17:07:47.464085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-14 17:07:47.464142: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-14 17:07:47.464185: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-14 17:07:47.464228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [9]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    verbose=1,
    shuffle=False
)

Epoch 1/50
597/597 [==============================] - 29s 47ms/step - loss: 1.6713 - mse: 1.6713
Epoch 2/50
597/597 [==============================] - 28s 47ms/step - loss: 1.5251 - mse: 1.5251
Epoch 3/50
597/597 [==============================] - 28s 47ms/step - loss: 1.5389 - mse: 1.5389
Epoch 4/50
597/597 [==============================] - 28s 47ms/step - loss: 1.5374 - mse: 1.5374
Epoch 5/50
597/597 [==============================] - 28s 48ms/step - loss: 1.5423 - mse: 1.5423
Epoch 6/50
597/597 [==============================] - 29s 48ms/step - loss: 1.5421 - mse: 1.5421
Epoch 7/50
597/597 [==============================] - 29s 48ms/step - loss: 1.5420 - mse: 1.5420
Epoch 8/50
597/597 [==============================] - 30s 50ms/step - loss: 1.5421 - mse: 1.5421
Epoch 9/50
597/597 [==============================] - 33s 56ms/step - loss: 1.5415 - mse: 1.5415
Epoch 10/50
597/597 [==============================] - 35s 59ms/step - loss: 1.5415 - mse: 1.5415
Epoch 11/50
597/597 [========

In [10]:
y_pred = model.predict(X_val)
y_pred

array([[2.6166413]], dtype=float32)

# Model saving

In [13]:
model.save("./models/LSTM_multi_with_target.h5", include_optimizer=True, save_format="h5")